In [1]:
%matplotlib notebook
%matplotlib inline

In [1]:
!pip3 install torch torchvision torchaudio

Defaulting to user installation because normal site-packages is not writeable


In [7]:
#go to yolov5 folder and install requirements.txt 
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
import tqdm
# install yolov5 then other libraries will solve start
sys.path.append('D:\Shravtek\Engraving_text_ocr\engraved_ocr_complete_source\yolov5')
import torch
from models.common import DetectMultiBackend
from utils.torch_utils import select_device
from utils.dataloaders import LoadImages
from utils.general import non_max_suppression,Profile,scale_boxes,xyxy2xywh
from pathlib import Path
from utils.plots import Annotator
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
# end


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for gitpython>=3.1.30 from https://files.pythonhosted.org/packages/8d/c4/82b858fb6483dfb5e338123c154d19c043305b01726a67d89532b8f8f01b/GitPython-3.1.40-py3-none-any.whl.metadata
  Obtaining dependency information for opencv-python>=4.1.1 from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
  Using cached opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached thop-0.1.1.post2209072238-py3-none-any.whl (15 kB)
  Obtaining dependency information for ultralytics>=8.0.147 from https://files.pythonhosted.org/packages/9a/60/2bfc23a2ba1819305ccea076fdd8d8cd67190ab9d20a9838e5b7b731367c/ultralytics-8.0.209-py3-none-any.whl.metadata
  Obtaining dependency information for gitdb<5,>=4.0.1 from https://files.pythonhosted.org/packages/fd/5b/8f0c4a5bb9fd4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import cv2
from skimage import filters

from skimage import feature
import numpy as np
from scipy.ndimage import distance_transform_edt
import os
import pandas as pd
import glob
from scipy.ndimage import label, binary_fill_holes

import string

from PIL import Image
from torchvision.transforms import ToTensor,Resize

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

weights = 'D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/best.pt'
dnn= False
fp16=False
data = 'D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/data.yaml'
model = DetectMultiBackend(weights, device=device, dnn=dnn, data=data, fp16=fp16)
imgsz=(640, 640)
pt = model.pt
model.warmup(imgsz=(1 if pt or model.triton else bs, 3, *imgsz))

Fusing layers... 
Model summary: 322 layers, 86180143 parameters, 0 gradients, 203.8 GFLOPs


In [12]:
device

device(type='cpu')

In [13]:
def get_yolo_results(image_path,model):
    source = image_path
    imgsz=(640, 640)
    stride = model.stride
    pt = model.pt
    dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt)
    conf_thres = 0.25
    iou_thres=0.45
    classes = None
    agnostic_nms = False
    max_det = 1000
    webcam = False
    save_dir = Path('D:/Upwork/KS/MOHIT/')
    save_crop = False
    line_thickness = 3
    names = model.names

    total_detections = []
    seen, windows, dt = 0, [], (Profile(), Profile(), Profile())
    for path, im, im0s, vid_cap, s in dataset:
            with dt[0]:
                im = torch.from_numpy(im).to(model.device)
                im = im.half() if model.fp16 else im.float()  # uint8 to fp16/32
                im /= 255  # 0 - 255 to 0.0 - 1.0
                if len(im.shape) == 3:
                    im = im[None]  # expand for batch dim

            # Inference
            with dt[1]:
                visualize = False
                augment = False
                pred = model(im, augment=augment, visualize=visualize)

            # NMS
            with dt[2]:
                pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
            for i, det in enumerate(pred):  # per image
                seen += 1
                if webcam:  # batch_size >= 1
                    p, im0, frame = path[i], im0s[i].copy(), dataset.count
                    s += f'{i}: '
                else:
                    p, im0, frame = path, im0s.copy(), getattr(dataset, 'frame', 0)

                p = Path(p)  # to Path
                save_path = str(save_dir / p.name)  # im.jpg
                #txt_path = str(save_dir / 'labels' / p.stem) + ('' if dataset.mode == 'image' else f'_{frame}')  # im.txt
                txt_path = str(save_dir / p.stem)
                s += '%gx%g ' % im.shape[2:]  # print string
                gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                imc = im0.copy() if save_crop else im0  # for save_crop
                annotator = Annotator(im0, line_width=line_thickness, example=str(names))
                if len(det):
                    # Rescale boxes from img_size to im0 size
                    det[:, :4] = scale_boxes(im.shape[2:], det[:, :4], im0.shape).round()

                    # Print results
                    for c in det[:, 5].unique():
                        n = (det[:, 5] == c).sum()  # detections per class
                        s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                    # Write results
                    save_conf = True
                    for *xyxy, conf, cls in reversed(det):
                            xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                            line = (cls, *xywh, conf) if save_conf else (cls, *xywh)  # label format
    #                         with open(f'{txt_path}.txt', 'a') as f:
    #                             f.write(('%g ' * len(line)).rstrip() % line + '\n')
                            total_detections.append(line)
        
    converted_detections = [tuple(value.item() if isinstance(value, torch.Tensor) else value for value in item) for item in total_detections]
        
    return converted_detections

In [15]:
%%time
get_yolo_results('D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/images/00003_1007B/IMG20230717185759.jpg',model)

CPU times: total: 4 s
Wall time: 1.14 s


[(1.0,
  0.4954427182674408,
  0.6581307649612427,
  0.2200520783662796,
  0.1032986119389534,
  0.8900173306465149),
 (0.0,
  0.5552300214767456,
  0.24334490299224854,
  0.2345920205116272,
  0.10995370149612427,
  0.9114524722099304)]

In [16]:
def get_bboxes(image_path, detections):
    image = cv2.imread(image_path)
    
    bounding_boxes = []
    
    for cls, x, y, w, h, confidence in detections:
        cls = int(cls)
        left = int((x - w/2) * image.shape[1]) - 10
        top = int((y - h/2) * image.shape[0]) - 10
        width = int(w * image.shape[1]) + 20
        height = int(h * image.shape[0]) + 20
        
        bounding_boxes.append({
            'class': cls,
            'left': left,
            'top': top,
            'width': width,
            'height': height
        })
    
    return bounding_boxes

In [18]:
detections = get_bboxes('D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/images/00003_1007B/IMG20230717185759.jpg',
                        get_yolo_results('D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/images/00003_1007B/IMG20230717185759.jpg'
                                         ,model))

In [19]:
detections

[{'class': 1, 'left': 1766, 'top': 2085, 'width': 1033, 'height': 377},
 {'class': 0, 'left': 2007, 'top': 640, 'width': 1101, 'height': 399}]

In [25]:
img = cv2.imread('D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/images/00003_1007B/IMG20230717185759.jpg')
cropped_img = img[detections[1]['top']:detections[1]['top']+detections[1]['height'],detections[1]['left']:detections[1]['left']+detections[1]['width']]
plt.imshow(cropped_img)
plt.show()

In [28]:
def calculate_bbox_area(bbox):
    width = bbox['width']
    height = bbox['height']
    return width * height

def remove_lowest_area_bbox(detections):
    if len(detections) <= 2:
        return detections

    areas = [calculate_bbox_area(bbox) for bbox in detections]
    min_area_index = areas.index(min(areas))
    
    # Return detections without the detection with the lowest area
    return detections[:min_area_index] + detections[min_area_index + 1:]


In [29]:
remove_lowest_area_bbox(detections)

[{'class': 1, 'left': 1766, 'top': 2085, 'width': 1033, 'height': 377},
 {'class': 0, 'left': 2007, 'top': 640, 'width': 1101, 'height': 399}]

In [34]:
# Define the path to the checkpoint
checkpoint_path = 'D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/epoch=16-step=2086-val_accuracy=100.0000-val_NED=100.0000.ckpt'

# Define the model parameters
model_args = {
    'data_root': 'data',
    'batch_size': 1,  # Set batch size to 1 for inference on singular images
    'num_workers': 4,
    'cased': False,
    'punctuation': False,
    'new': False,  # Set to True if you want to evaluate on new benchmark datasets
    'rotation': 0,
    'device': 'cpu'  # Use 'cuda' or 'cpu' depending on your environment
}


In [31]:
# Load the model checkpoint
model_ocr = torch.hub.load('baudm/parseq', 'parseq', pretrained=True,map_location=torch.device('cpu'))  # Example: Replace with your model loading code
model_ocr.load_state_dict(torch.load(checkpoint_path,map_location=torch.device('cpu'))['state_dict'])
model_ocr.eval()
model_ocr.to(model_args['device'])


Using cache found in C:\Users\shan/.cache\torch\hub\baudm_parseq_main


RuntimeError: Missing dependencies: pytorch_lightning, timm

In [32]:
pip install ultralytics dill pytorch-lightning==1.9.5 timm nltk

Defaulting to user installation because normal site-packages is not writeable
  Using cached pytorch_lightning-1.9.5-py3-none-any.whl (829 kB)
  Obtaining dependency information for timm from https://files.pythonhosted.org/packages/f0/1e/05287cb8984229d101874433df472b1fa3dcd6f746ccb6e8a26c7deeb1c7/timm-0.9.10-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/59.8 kB ? eta -:--:--
     ---------------------------------------- 59.8/59.8 kB 1.5 MB/s eta 0:00:00
  Obtaining dependency information for torchmetrics>=0.7.0 from https://files.pythonhosted.org/packages/a3/88/cc27059747ddecff744826e38014822023cbfff4ca079a6ee9a96602dd0b/torchmetrics-1.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for lightning-utilities>=0.6.0.post0 from https://files.pythonhosted.org/packages/46/ee/8641eeb6a062f383b7d6875604e1f3f83bd2c93a0b4dbcabd3150b32de6e/lightning_utilities-0.9.0-py3-none-any.whl.metadata
  Using cached lightning_utilities-0.9.0-py3-none-any.whl.m

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [35]:
# Load the model checkpoint
model_ocr = torch.hub.load('baudm/parseq', 'parseq', pretrained=True,map_location=torch.device('cpu'))  # Example: Replace with your model loading code
model_ocr.load_state_dict(torch.load(checkpoint_path,map_location=torch.device('cpu'))['state_dict'])
model_ocr.eval()
model_ocr.to(model_args['device'])


Using cache found in C:\Users\shan/.cache\torch\hub\baudm_parseq_main


PARSeq(
  (encoder): Encoder(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(4, 8), stride=(4, 8))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (drop1)

In [36]:
def perform_inference(model, image):
    transform = ToTensor()
    resize = resize = Resize((32, 128))   # Resize the image to match the model's input size
    image = Image.fromarray(image)
    image = image.convert("RGB")
    image = resize(image)  # Resize the image
    image_tensor = transform(image).unsqueeze(0).to(model_args['device'])

    with torch.no_grad():
        output = model(image_tensor)
        # Process the output as needed

    return output

def ocr_text(model,image):
    inference_result = perform_inference(model, image)
    # Greedy decoding
    pred = inference_result.softmax(-1)
    label, confidence = model_ocr.tokenizer.decode(pred)
    return (label[0],["{:.2%}".format(value) for value in confidence[0].tolist()[:-1]])

In [42]:
img = cv2.imread('D:/Shravtek/Engraving_text_ocr/engraved_ocr_complete_source/images/synthetic_dataset/0-500/1-100/94.jpg')
plt.imshow(img)
plt.show()x

In [43]:
ocr_text(model_ocr,img)

('00094', ['100.00%', '100.00%', '100.00%', '100.00%', '100.00%'])

###### Testing Model on all the images

In [22]:
filenames_list = []
expected_Dot_Peening_Text_list = []
predicted_Dot_Peening_Text_list = []
expected_Machine_Peening_Text_list = []
predicted_Machine_Peening_Text_list = []
for curdir in tqdm.tqdm(os.listdir('/home/ubuntu/mohit/OCR-FineTuning/parseq/data/data2/actual_cam/')):
    for curfile in tqdm.tqdm(glob.glob(f'/home/ubuntu/mohit/OCR-FineTuning/parseq/data/data2/actual_cam/{curdir}/*.jpg')):
        img = img = cv2.imread(curfile)
        expected_texts = curdir.split('_')
        detections = get_bboxes(curfile, get_yolo_results(curfile,model))
#         if len(detections) > 2:
#             break
        detections = remove_lowest_area_bbox(detections)
        for detection in detections:
            if detection['class'] == 0:
                cropped_img = img[detection['top']:detection['top']+detection['height'],detection['left']:detection['left']+detection['width']]
                predicted_Dot_Peening_Text_list.append(ocr_text(model_ocr,cropped_img))
                #predicted_Dot_Peening_Text_list.append(ocr.ocr(sharpen_image(cropped_img))[0][0][1][0].replace(' ',''))
            else:
                cropped_img = img[detection['top']:detection['top']+detection['height'],detection['left']:detection['left']+detection['width']]
                predicted_Machine_Peening_Text_list.append(ocr_text(model_ocr,cropped_img))
        
        filenames_list.append(curfile)
        expected_Dot_Peening_Text_list.append(expected_texts[1])
        expected_Machine_Peening_Text_list.append(expected_texts[0])
        break
    break
    

  0%|                                                                                                               | 0/6 [00:01<?, ?it/s]


In [23]:
df = pd.DataFrame({'filenames':filenames_list,'expected_Dot_Peening':expected_Dot_Peening_Text_list,
                  'predicted_Dot_Peening':predicted_Dot_Peening_Text_list,
                   'expected_Machine_Peening':expected_Machine_Peening_Text_list,
                   'predicted_Machine_Peening':predicted_Machine_Peening_Text_list})
df

,filenames,expected_Dot_Peening,predicted_Dot_Peening,expected_Machine_Peening,predicted_Machine_Peening
0,/home/ubuntu/mohit/OCR-FineTuning/parseq/data/...,00324,00324,1006C1,1006C1


In [24]:
# Create comparison columns
df['Dot_Peening_Match'] = df.apply(
    lambda row: row['expected_Dot_Peening'] == row['predicted_Dot_Peening'],
    axis=1
)

df['Machine_Peening_Match'] = df.apply(
    lambda row: row['expected_Machine_Peening'] == row['predicted_Machine_Peening'],
    axis=1
)

# Create an overall match column
df['Overall_Match'] = df['Dot_Peening_Match'] & df['Machine_Peening_Match']
df['Overall_Match'].value_counts()

Overall_Match
True    1
Name: count, dtype: int64

In [71]:
df.dtypes

filenames                    object
expected_Dot_Peening         object
predicted_Dot_Peening        object
expected_Machine_Peening     object
predicted_Machine_Peening    object
Dot_Peening_Match              bool
Machine_Peening_Match          bool
Overall_Match                  bool
dtype: object

In [70]:
df.to_csv('latest_results.csv',index=False)